# 서울시 도시 데이터 전처리

In [2]:
# 옵션 설정
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

# 필요한 패키지
import re
import requests
import numpy as np
import pandas as pd

In [3]:
with open("src/coord.json", "r") as f:
    coord = json.load(f)

In [4]:
len(coord['features'])
# 115개의 장소 위치 데이터

115

In [5]:
city = coord['features']

In [16]:
for _ in range(5):
    print(city[_]['properties']['hotspotNm'], end=" / ")
    print(city[_]['properties']['category'], end="/ ")
    print(city[_]['properties']['center'])
    # print(city[_]['geometry']['coordinates'][0], "\n")

홍대 관광특구 / 관광특구/ [37.55458049807929, 126.92187309265138]
창덕궁·종묘 / 고궁·문화유산/ [37.57927647015724, 126.99345588684083]
서울역 / 인구밀집지역/ [37.55662179786924, 126.9728994369507]
가산디지털단지역 / 인구밀집지역/ [37.48088619499635, 126.87994480133058]
건대입구역 / 인구밀집지역/ [37.53967731569061, 127.06872940063478]


In [17]:
AREA_NM = []
CENTER = []
COORDINATES = []

for _ in range(len(city)):
    AREA_NM.append(city[_]['properties']['hotspotNm'])
    CENTER.append(city[_]['properties']['center'])
    COORDINATES.append(city[_]['geometry']['coordinates'][0])

In [18]:
city_df = pd.DataFrame(
    {"AREA_NM" : AREA_NM,
     "CENTER" : CENTER,
     "COORDINATES" : COORDINATES
     })
city_df.head(3)

,AREA_NM,CENTER,COORDINATES
0,홍대 관광특구,"[37.55458049807929, 126.92187309265138]","[[37.54899139823932, 126.91366876415408], [37...."
1,창덕궁·종묘,"[37.57927647015724, 126.99345588684083]","[[37.583318, 126.989366], [37.583408, 126.9893..."
2,서울역,"[37.55662179786924, 126.9728994369507]","[[37.555967, 126.976832], [37.555829, 126.9767..."


In [19]:
city_code = pd.read_excel("src/city_code.xlsx")
city_code_df = city_code.iloc[:,[0,2,3]]

In [20]:
city_code_df.head(3)

,CATEGORY,AREA_CD,AREA_NM
0,관광특구,POI001,강남 MICE 관광특구
1,관광특구,POI002,동대문 관광특구
2,관광특구,POI003,명동 관광특구


In [21]:
city_data_df = pd.merge(city_code_df, city_df, on="AREA_NM", how="inner")

In [22]:
city_data_df.head(3)

,CATEGORY,AREA_CD,AREA_NM,CENTER,COORDINATES
0,관광특구,POI001,강남 MICE 관광특구,"[37.51095136635107, 127.06016778945924]","[[37.513903342198184, 127.06027547260382], [37..."
1,관광특구,POI002,동대문 관광특구,"[37.567269671705716, 127.0113515853882]","[[37.56949795219521, 127.01759028830955], [37...."
2,관광특구,POI003,명동 관광특구,"[37.564650367842184, 126.98208332061769]","[[37.56066567164939, 126.97600605147264], [37...."


In [23]:
HASHTAG = []

for _ in city_data_df['AREA_NM']:
    if _ == "4·19 카페거리":
        formatted_tokens = [_]
    else:
        tokens = re.findall(r'\w+|\d[·\d]*|[^\W\d_]+(?:\([^\)]+\))?', _)
        formatted_tokens = [re.sub(r'·', ',', token).strip() for token in tokens]
    HASHTAG.append(formatted_tokens[0])

In [24]:
city_data_df.insert(3, "HASHTAG", HASHTAG)

In [25]:
city_data_df.head(3)

,CATEGORY,AREA_CD,AREA_NM,HASHTAG,CENTER,COORDINATES
0,관광특구,POI001,강남 MICE 관광특구,강남,"[37.51095136635107, 127.06016778945924]","[[37.513903342198184, 127.06027547260382], [37..."
1,관광특구,POI002,동대문 관광특구,동대문,"[37.567269671705716, 127.0113515853882]","[[37.56949795219521, 127.01759028830955], [37...."
2,관광특구,POI003,명동 관광특구,명동,"[37.564650367842184, 126.98208332061769]","[[37.56066567164939, 126.97600605147264], [37...."


In [85]:
city_data_df.to_csv("src/city_data_df.csv", index=False)